In [72]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [73]:
import numpy as np

In [74]:
import pandas as pd

In [75]:
#in kaggle - /kaggle/input/simplesentiment/products_sentiment_train.tsv
train_data = pd.read_csv('products_sentiment_train.tsv', sep='\t', header=None)

In [76]:
train_data.head()

,0,1
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [77]:
#in kaggle - /kaggle/input/simplesentiment/products_sentiment_sample_submission.csv
sample_submission = pd.read_csv('products_sentiment_sample_submission.csv')

In [78]:
sample_submission.head()

,Id,y
0,0,0
1,1,1
2,2,0
3,3,1
4,4,0


In [79]:
#in kaggle - /kaggle/input/simplesentiment/products_sentiment_test.tsv
test_data = pd.read_csv('products_sentiment_test.tsv', sep='\t')

In [80]:
test_data.head()

,Id,text
0,0,"so , why the small digital elph , rather than ..."
1,1,3/4 way through the first disk we played on it...
2,2,better for the zen micro is outlook compatibil...
3,3,6 . play gameboy color games on it with goboy .
4,4,"likewise , i 've heard norton 2004 professiona..."


In [81]:
train_data.shape, test_data.shape, sample_submission.shape

((2000, 2), (500, 2), (500, 2))

In [82]:
X_train_words = np.array(train_data[0])

In [83]:
y_train = np.array(train_data[1])

In [85]:
X_test_words = np.array(test_data['text'])

In [88]:
y_test = np.array(sample_submission['y'])

In [89]:
X_train_words.shape, y_train.shape, X_test_words.shape, y_test.shape

((2000,), (2000,), (500,), (500,))

Как и в предыдущем задании проверим качество кросс-валидации несколькоих алгоритмов: алгоритмов векторизации TfidfVectoriser и CountVectorizer и несколько моделей классификации - LogisticRegression, LinearSVC, SGDClassifier. Для этого пройдемся циклом по нескольким пайплайнам, где методы будут со стандартными настройками и оценим качество каждого. Потом метод с наилучшим качеством будем доработывать до достижения байзлайна. 

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [91]:
from sklearn.svm import LinearSVC

In [92]:
from sklearn.linear_model import SGDClassifier

In [93]:
from sklearn.linear_model import LogisticRegression

In [94]:
from sklearn.pipeline import Pipeline

In [95]:
from sklearn.model_selection import cross_val_score

In [96]:
methods_accuracy = []

In [97]:
for vectorizer in [TfidfVectorizer(), CountVectorizer()]:
  for classifier in [LinearSVC(), SGDClassifier(), LogisticRegression()]:
    pipe = Pipeline([ ('vectorizer', vectorizer), ('classifier', classifier) ])
    answer = cross_val_score(pipe, X_train_words, y_train).mean()
    methods_accuracy.append( (str(vectorizer), str(classifier), answer) )

In [98]:
methods_accuracy

[('TfidfVectorizer()', 'LinearSVC()', 0.7689999999999999),
 ('TfidfVectorizer()', 'SGDClassifier()', 0.7495),
 ('TfidfVectorizer()', 'LogisticRegression()', 0.766),
 ('CountVectorizer()', 'LinearSVC()', 0.754),
 ('CountVectorizer()', 'SGDClassifier()', 0.7460000000000001),
 ('CountVectorizer()', 'LogisticRegression()', 0.7685)]

Как мы видим, наибольшее качество у комбинациии TfidfVectorizer и LinearSVC. С ней и будем работать, добавим некоторые параметры в методы. 
В TFidfVectorizer добавим analyzer='char_wb'. В LinearSVC изменим регуляризатор с дефолтного L2 на L1, ограничим число итераций до 10 тысяч

In [153]:
pipe_with_params = Pipeline([ ('TfidfVectorizer', TfidfVectorizer(analyzer='char_wb')), 
                             ('LinearSVC', LinearSVC(penalty='l1', fit_intercept=False, dual=False, max_iter=100000)) ])

In [154]:
cross_val_score(pipe_with_params, X_train_words, y_train).mean()

0.6679999999999999

Как мы видим качество даже упало. Попробуем ввести ограничение на длину n-грамм из букв. Сначал зададим любой интервал и посмотрим, улучшится ли качество. Вместе с тем уберем все параметры из LinearSVC, возможно они послужили причиной снижения качества.

In [159]:
pipe_n_grams = Pipeline([ ('TfidfVectorizer', TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 10))), 
                             ('LinearSVC', LinearSVC()) ])

In [160]:
cross_val_score(pipe_n_grams, X_train_words, y_train).mean()

0.769

Видно, что мы повысили качество даже лучше первоначального значения!
Поиграемся со длиной n_грамм:

In [162]:
pipe_n_grams = Pipeline([ ('TfidfVectorizer', TfidfVectorizer(analyzer='char_wb', ngram_range=(5, 10))), 
                             ('LinearSVC', LinearSVC()) ])

In [163]:
cross_val_score(pipe_n_grams, X_train_words, y_train).mean()

0.766

In [164]:
pipe_n_grams = Pipeline([ ('TfidfVectorizer', TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 8))), 
                             ('LinearSVC', LinearSVC()) ])

In [165]:
cross_val_score(pipe_n_grams, X_train_words, y_train).mean()

0.772

In [176]:
pipe_n_grams = Pipeline([ ('TfidfVectorizer', TfidfVectorizer(analyzer='char_wb', ngram_range=(4, 7))), 
                             ('LinearSVC', LinearSVC()) ])

In [177]:
cross_val_score(pipe_n_grams, X_train_words, y_train).mean()

0.7775000000000001

Максимальное качество было достигнуто при длине n-грамм от 4 до 7. Теперь попробуем добавить стоп-слова по методу предыдущего задания

In [178]:
import nltk

In [179]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [195]:
stopwords = nltk.corpus.stopwords.words('english')

In [196]:
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

Добавим список стоп-слов как входной параметр TfidfVectorizer:

In [197]:
pipe_stop = Pipeline([ ('TfidfVectorizer', TfidfVectorizer(analyzer='char_wb', ngram_range=(4, 7), stop_words=stopwords)), 
                             ('LinearSVC', LinearSVC()) ])

In [198]:
cross_val_score(pipe_stop, X_train_words, y_train).mean()

0.7775000000000001

Добавим к списку стоп-биграммы:

In [199]:
bigrams = list(nltk.bigrams(stopwords))

In [200]:
for i in bigrams:
  stopwords.append(' '.join(i))

In [201]:
cross_val_score(pipe_stop, X_train_words, y_train).mean()

0.7775000000000001

Как видно, ни использование стоп-слов, ни их биграмм не помогло улучшить качество.

Используем для тестовых данных пайплайн pipe_n_grams и сохраним данные в файл csv.

In [244]:
pipe_n_grams.fit(X_train_words, y_train)

Pipeline(steps=[('TfidfVectorizer',
                 TfidfVectorizer(analyzer='char_wb', ngram_range=(4, 7))),
                ('LinearSVC', LinearSVC())])

In [245]:
answer = pipe_n_grams.predict(X_test_words)

In [247]:
output = pd.DataFrame( {'Id': np.arange(0, len(answer)), 'y': answer} )

In [248]:
output.head()

,Id,y
0,0,1
1,1,0
2,2,1
3,3,1
4,4,0


In [249]:
output.to_csv('submission.csv', index=False)